# 🐼 บทที่ 1: ปูพื้นฐาน Pandas สำหรับงาน ISP
---

**Pandas** (แพน-ดาส) คือ Library ที่เก่งที่สุดใน Python สำหรับการจัดการข้อมูลที่มีลักษณะเป็นตาราง (เหมือน Excel) ช่วยให้เราอ่านข้อมูล ISP หลักหมื่นหลักแสนแถวได้ในเสี้ยววินาที

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Library** | ไล-บรา-รี | ชุดเครื่องมือหรือโค้ดที่มีคนเขียนไว้ให้เราเรียกใช้ |
| **DataFrame** | ดาต้า-เฟรม | **ตารางข้อมูล** (มีทั้งแถวและคอลัมน์) |
| **Series** | ซี-รีส์ | **ข้อมูล 1 คอลัมน์** (ข้อมูลแนวตั้งชุดเดียว) |
| **Attribute** | แอท-ทริ-บิวต์ | คุณสมบัติประจำตัวของ Object (เช่น ชื่อชีท, จำนวนแถว) |

### 🛠️ เครื่องมือที่ต้องเตรียม
ก่อนเริ่ม ป๋าต้องมั่นใจว่ามี Library เหล่านี้ในเครื่อง:
1. `pandas`: ตัวจัดการตารางข้อมูล
2. `openpyxl`: ตัวช่วยให้ Python อ่านไฟล์ `.xlsx` ได้

In [ ]:
# ถ้าป๋ายังไม่มี ให้รันบรรทัดนี้ครับ (ใน Notebook ใส่ ! ข้างหน้าด้วย)
!pip install pandas openpyxl

## 📂 การเปิดไฟล์ด้วย ExcelFile (The Master Handler)
เราจะใช้ `pd.ExcelFile` เพราะมันเหมาะกับงาน ISP ที่มีหลายชีท ช่วยให้เราเปิดไฟล์ทิ้งไว้ครั้งเดียวแล้วเลือกดึงข้อมูลได้ตามใจชอบ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Object** | ออบ-เจกต์ | วัตถุที่สร้างขึ้นในโปรแกรมเพื่อจัดการงานบางอย่าง |
| **Parse** | พาร์ซ | การอ่านและตีความข้อมูลจากไฟล์ออกมาเป็นตาราง |
| **Sheet Names** | ชีท เนมส์ | รายชื่อแผ่นงานที่มีอยู่ในไฟล์ Excel |

In [ ]:
import pandas as pd

# 1. โหลดไฟล์เข้ามาเป็น Object
# เปรียบเหมือนป๋าหยิบสมุดบัญชี ISP มาวางบนโต๊ะ
excel_obj = pd.ExcelFile('mock_isp_data.xlsx')

# 2. ตรวจดูว่าในสมุดเล่มนี้มีชีทชื่ออะไรบ้าง
print(f'รายชื่อชีททั้งหมด: {excel_obj.sheet_names}')

# 3. ดึงข้อมูลจากชีท 'Main_Data' ออกมาเป็นตาราง (DataFrame)
# และบังคับให้ MSISDN เป็นข้อความ (str) ทันทีเพื่อความสวยงาม
df = excel_obj.parse('Main_Data', dtype={'MSISDN': str})

# 4. ดูหน้าตาข้อมูล 5 แถวแรก
df.head()

## 🔍 การสำรวจข้อมูล (Data Inspection)
เมื่อได้ตารางมาแล้ว เราต้อง "ส่อง" ดูคุณภาพข้อมูลก่อนเริ่มงานจริง

* `df.head()` : ขอดู 5 แถวแรก
* `df.info()` : ดูโครงสร้างตาราง (จำนวนแถว, ประเภทข้อมูล, ค่าว่าง)
* `df.shape` : ดูขนาดของตาราง (กี่แถว x กี่คอลัมน์)

In [ ]:
# ดูประเภทข้อมูลของแต่ละคอลัมน์
# ป๋าจะเห็นว่า MSISDN เป็น 'object' (ข้อความ) ตามที่เราตั้งใจไว้
df.info()

# ดูขนาดของไฟล์ ISP ชุดนี้
rows, cols = df.shape
print(f'ไฟล์นี้มีข้อมูลทั้งหมด {rows} รายการ และมี {cols} หัวข้อ')

## 🗓️ การแปลงวันที่หลายคอลัมน์พร้อมกัน (Bulk Datetime Conversion)
---

ในงาน ISP ป๋ามักจะเจอคอลัมน์วันที่เยอะมาก เช่น `วันที่เริ่มสัญญา`, `วันที่สิ้นสุด`, `วันที่ติดตั้ง` การมานั่งแปลงทีละบรรทัดจะทำให้โค้ดดูรกและเสียเวลาครับ เราจึงต้องใช้เทคนิคที่เรียกว่า **Vectorization**

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Vectorization** | เวก-เตอ-ไร-เซ-ชัน | การสั่งงานให้คอมพิวเตอร์ทำกับข้อมูลทั้งชุดพร้อมกัน (เร็วกว่าเขียน Loop) |
| **Bulk** | บัลค์ | ปริมาณมาก หรือการทำพร้อมกันทีละเยอะ ๆ |
| **Apply** | อะ-พลาย | การประยุกต์ใช้ฟังก์ชันหนึ่งกับทุก ๆ สมาชิกในกลุ่มที่เราเลือก |
| **List** | ลิสต์ | รายการสิ่งของที่รวบรวมไว้ (ใน Python เขียนใน `[ ]`) |

### 💡 หลักการทำงาน
เราจะสร้าง **List** ของชื่อคอลัมน์ที่ต้องการ แล้วใช้คำสั่ง `.apply(pd.to_datetime)` เพื่อสั่งให้ Pandas วิ่งไปแปลงทุกคอลัมน์ใน List นั้นให้เป็นวันที่ในคำสั่งเดียวครับ

In [ ]:
# 1. สมมติป๋ามีลิสต์ชื่อคอลัมน์วันที่ (ป๋าเพิ่มชื่อเข้าไปในนี้ได้เรื่อยๆ เลย)
date_cols = [
    'CONTRACT_START_DT',
    'CONTRACT_END_DT',
    # 'INSTALL_DATE'          # สมมติว่ามีคอลัมน์นี้เพิ่มมา
]
# 2. ใช้เทคนิค Bulk Conversion (แปลงรวดเดียว)
# เราเลือกเฉพาะคอลัมน์ในลิสต์ แล้วสั่ง apply ฟังก์ชัน to_datetime
df[date_cols] = df[date_cols].apply(pd.to_datetime)

# 3. เช็กผลลัพธ์ด้วย .info()
# ป๋าจะเห็นว่าคอลัมน์ในลิสต์เปลี่ยนเป็น datetime64[ns] ทั้งหมดแล้วครับ
df.info()


### ⚠️ ทริคเสริม: ถ้าวันที่มาในรูปแบบแปลกๆ
บางที ISP ส่งไฟล์มาเป็นแบบ `31-12-2024` หรือ `2024/12/31` ป๋าสามารถบังคับ **Format** (ฟอร์-แมต) เพื่อความแม่นยำได้ครับ

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Format String** | ฟอร์-แมต สตริง | รหัสที่ใช้บอกคอมพิวเตอร์ว่า วัน/เดือน/ปี อยู่ตรงไหน (เช่น `%d/%m/%Y`) |
| **Ambiguous** | แอม-บิ-กิว-อัส | กำกวม (เช่น 01/02 อาจหมายถึง 1 กุมภาพันธ์ หรือ 2 มกราคม) |

> **คำแนะนำ:** ถ้าป๋ารู้รูปแบบแน่นอน ให้ใส่ `format` กำกับไว้เสมอ จะช่วยลดความผิดพลาดจากความกำกวมได้ครับ

In [ ]:
df[date_cols] = df[date_cols].apply(pd.to_datetime, format='%Y-%m-%d')

df[date_cols].head()
# df.info()

## ✂️ การตัด "เวลา" ออกจาก "วันที่" (Removing Time from Datetime)
---

โดยปกติเมื่อเราใช้ `pd.to_datetime` ข้อมูลจะถูกเก็บในรูปแบบ `YYYY-MM-DD HH:MM:SS` (มีเวลาพ่วงมาด้วย) หากป๋าต้องการแค่ส่วนของวันที่ เรามี 2 ทางเลือกหลักครับ:

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **Accessor (.dt)** | แอ็ก-เซส-เซอร์ | ตัวช่วยในการเข้าถึงคุณสมบัติพิเศษของข้อมูลประเภทวันที่ |
| **Normalize** | นอร์-มะ-ไลซ์ | การปรับค่าให้เป็นมาตรฐาน (เช่น ปรับเวลาให้เป็น 00:00:00 ทั้งหมด) |
| **String Formatting** | สตริง ฟอร์-แมต-ติง | การแปลงวันที่ให้เป็น "ข้อความ" ตามรูปแบบที่เรากำหนด |
| **Date Object** | เดท ออบ-เจกต์ | ข้อมูลที่มีเฉพาะ ปี-เดือน-วัน (ไม่มีเวลา) |

### 💡 วิธีที่ป๋าควรเลือก
1. **ใช้ `.dt.date`:** ถ้าป๋าต้องการแค่ "ค่า" ไปใช้งานต่อ (แต่ Type จะกลายเป็น Object/Date)
2. **ใช้ `.dt.strftime`:** ถ้าป๋าต้องการความสวยงามในรายงาน (Type จะกลายเป็น String)

In [ ]:
df[date_cols] = df[date_cols].apply(lambda x: x.dt.date)

display(df[date_cols])

### ✨ การแต่งตัวให้วันที่ (Formatting for Presentation)
หากป๋าต้องเอาตารางนี้ไปทำรายงานต่อ หรือก๊อปไปวางในแอปฯ อื่น ป๋าอาจจะอยากเห็นวันที่ในรูปแบบที่คุ้นเคย เช่น `31/12/2024`

* `%d` = วัน (01-31)
* `%m` = เดือน (01-12)
* `%Y` = ปี ค.ศ. (2024)

> **คำเตือนของจุกทอง:** ถ้าป๋าใช้ `strftime` (แปลงเป็นข้อความ) ป๋าจะเอาไปบวกลบหาวันที่คงเหลือไม่ได้ทันทีนะป๋า ต้องทำหลังจากคำนวณเสร็จแล้วครับ!

In [ ]:
# --- วิธีที่ 2: แปลงเป็นข้อความเพื่อความสวยงาม (รายงาน) ---
# รูปแบบ วัน/เดือน/ปี
# show_date = ['CONTRACT_START_DT', 'CONTRACT_END_DT']
df['SHOW_DATE'] = df['CONTRACT_START_DT'].dt.strftime('%d/%m/%Y')

df['SHOW_DATE'].head()

## ❌ ทำไมถึงเกิด AttributeError?
---

ปัญหาที่ป๋าเจอเรียกว่า **"Type Conflict"** (การขัดกันของชนิดข้อมูล) ครับ

1. **ตอนแรก:** คอลัมน์เป็น `datetime64` (มีพลังพิเศษ `.dt` ใช้จัดรูปแบบได้)
2. **พอใช้ `.dt.date`:** Pandas จะถอดพลังพิเศษออก และเปลี่ยนให้เป็น `object` (เก็บแค่ค่าวันที่เฉยๆ) เพื่อตัดเวลาทิ้ง
3. **พอจะใช้ `.dt.strftime`:** Pandas เลยด่าว่า "ป๋าครับ คอลัมน์นี้ไม่มีพลัง `.dt` แล้วนะ!"

| คำศัพท์ (Technical Term) | คำอ่านไทย (Pronunciation) | ความหมาย (Translation/Definition) |
| :--- | :--- | :--- |
| **AttributeError** | แอท-ทริ-บิวต์ เออร์-เรอร์ | ข้อผิดพลาดจากการเรียกใช้ความสามารถที่ "ไม่มีอยู่" ในตัวแปรนั้น |
| **Datetimelike** | เดท-ไทม์-ไลค์ | ข้อมูลที่มีคุณสมบัติเป็นวันที่และเวลาที่ Pandas ยอมรับ |
| **Mutation** | มิว-เท-ชัน | การกลายพันธุ์ หรือการเปลี่ยนสภาพของข้อมูล |
| **Conflict** | คอน-ฟลิกต์ | การขัดกัน หรือการเข้ากันไม่ได้ |

> **กฎเหล็กของจุกทอง:** ถ้าป๋าอยากใช้ `.dt` เรื่อย ๆ ห้ามใช้ `.dt.date` ทับคอลัมน์เดิมเด็ดขาด! ให้เก็บเป็น `datetime64` ไว้จนกว่าจะถึงขั้นตอนสุดท้ายครับ

สรุปเทคนิคสำหรับป๋า:

ถ้าป๋าจะเอาไป "คำนวณ" (เช่น หาวันที่เหลือ) -> ให้คงสถานะ Datetime หรือใช้ .dt.date

ถ้าป๋าจะเอาไป "โชว์" (ทำรายงานให้สวยงาม) -> ให้ใช้ .dt.strftime('%d/%m/%Y')

ตอนนี้วันที่ของป๋าดู "คลีน" ขึ้นหรือยังครับ? ถ้าป๋าอยากลองทำโจทย์ยากขึ้นอย่าง "หาวันที่เหลือจนกว่าจะหมดสัญญา" (เช่น เหลืออีก 45 วัน) บอกจุกทองได้เลยนะ เดี๋ยวจุกทองพาเขียน Logic ครับ!

In [ ]:
# --- เริ่มใหม่แบบถูกวิธี ---

# 1. แปลงเป็น datetime64 ก่อน (พลัง .dt จะยังอยู่ครบ)
df[date_cols] = df[date_cols].apply(pd.to_datetime)

# 2. สร้างคอลัมน์สำหรับโชว์ (ทำตอนที่ยังมีพลัง .dt อยู่)
# วิธีนี้จะได้วันที่สวยๆ โดยที่คอลัมน์หลักยังเอาไปคำนวณได้
df['SHOW_DATE'] = df['CONTRACT_START_DT'].dt.strftime('%d/%m/%Y')
display(df['SHOW_DATE'])
# 3. ถ้าป๋าอยากให้ "หน้าตา" ในตารางหลักไม่มีเวลา (แต่ยังเป็น datetime อยู่)
# ให้ใช้ .dt.normalize() แทนการใช้ .dt.date ครับ
df[date_cols] = df[date_cols].apply(lambda x: x.dt.normalize())

# เช็กผลลัพธ์
display(df[date_cols].head())
df.info()


### 🛠️ ป๋าควรเลือกใช้ตัวไหนดี?

1. **`.dt.normalize()`**: ใช้เมื่อต้องการ **ตัดเวลาทิ้ง (ให้เป็น 00:00:00)** แต่ยังอยากให้เป็น `datetime64` เพื่อเอาไปบวกลบหาวันคงเหลือได้ (แนะนำวิธีนี้ครับ)
2. **`.dt.strftime()`**: ใช้เมื่อต้องการ **แต่งหน้าข้อมูล** ให้สวยเพื่อทำรายงาน (ผลลัพธ์จะเป็นข้อความ/String)
3. **`.dt.date`**: ใช้เมื่อต้องการ **ค่าวันที่เพียวๆ** จริงๆ และไม่คิดจะใช้พลังของ Pandas กับคอลัมน์นั้นอีกแล้ว

> **สรุป:** ถ้าป๋าจะทำงานต่อ ให้ใช้ `normalize()` แทน `date` ครับชีวิตจะง่ายขึ้นเยอะ!